In [248]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
pd.set_option('display.max_rows', 500)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Python Version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [249]:
# yymmdd format
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

# iterate through dates and pull in csv and cat dataframes together
df_turns = []
for date in dates:
    df_turns.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+date+'.txt'))
    
# Concat all dataframes
df_turns = pd.concat(df_turns)  

In [250]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names


In [251]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column?
del df_turns['DESC']

In [252]:
# Create a DateTime column that can be used for timeseries
df_turns.loc[:,'DATETIME'] = pd.to_datetime(df_turns['DATE'] + ' ' + df_turns['TIME'])

#Convert Date column to proper date type.
df_turns['DATE']  = pd.to_datetime(df_turns['DATE'],format='%m/%d/%Y')

#convert time to datetime type
df_turns['TIME']=pd.to_datetime(df_turns['TIME'], format='%H:%M:%S')

In [253]:
#backup
df_backup=df_turns.copy()

In [254]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [255]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [256]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

### Rami's Filtering

In [257]:
def Add_Weekday(data_frame, column='Date'):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame
df_turns_backup2 = df_turns.copy()

In [258]:
df_turns = Add_Weekday(df_turns, 'DATE')

In [259]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic,Day_Number,Weekday
2554,118307,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:24:57,65,30,2019-07-27 00:24:57,2.0,0.0,2.0,5,Sat
2592,118308,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:38:57,65,30,2019-07-27 00:38:57,0.0,0.0,0.0,5,Sat
2636,118309,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:52:57,65,30,2019-07-27 00:52:57,0.0,0.0,0.0,5,Sat
2640,118310,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:56:25,4043,3513,2019-07-27 00:56:25,3978.0,3483.0,7461.0,5,Sat
4489,118311,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 01:06:57,65,30,2019-07-27 01:06:57,3978.0,3483.0,7461.0,5,Sat


In [260]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

C:\Users\atag3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [355]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y
0,2019-07-27,1 AV,Sat,4308.0,5146.0,9454.0,0.95,40228.0
1,2019-07-28,1 AV,Sun,4106.0,5309.0,9415.0,0.95,40228.0
2,2019-07-29,1 AV,Mon,14483.0,16525.0,31008.0,0.95,40228.0
3,2019-07-30,1 AV,Tue,16375.0,18206.0,34581.0,0.95,40228.0
4,2019-07-31,1 AV,Wed,16812.0,19777.0,36589.0,0.95,40228.0


In [356]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [357]:
df_turns_no_outlier.sample(50)

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
17996,2019-09-16,MT EDEN AV,Mon,5302.0,3136.0,8438.0,0.95,9077.8,639.8
22283,2019-09-23,TWENTY THIRD ST,Mon,10527.0,12410.0,22937.0,0.95,36381.3,13444.3
5992,2019-08-06,7 AV,Tue,38542.0,36008.0,74550.0,0.95,83898.7,9348.7
18462,2019-08-09,NEVINS ST,Fri,9754.0,10030.0,19784.0,0.95,22373.7,2589.7
22619,2019-08-12,VERNON-JACKSON,Mon,14708.0,9830.0,24538.0,0.95,29941.4,5403.4
22732,2019-07-30,W 8 ST-AQUARIUM,Tue,3590.0,3056.0,6646.0,0.95,7740.2,1094.2
14474,2019-09-19,GRAND ST,Thu,33063.0,26234.0,59297.0,0.95,60969.4,1672.4
9482,2019-09-03,BEDFORD-NOSTRAN,Tue,8331.0,4597.0,12928.0,0.95,15425.0,2497.0
9162,2019-08-29,BEACH 67 ST,Thu,2611.0,1977.0,4588.0,0.95,4982.6,394.6
19381,2019-09-15,ORCHARD BEACH,Sun,0.0,0.0,0.0,0.95,185.2,185.2


# End of group data cleaning 

The data should now have outliers removed and can be aggregated as necessary for analysis. The last cell with save a pickled dataframe for easy import into other notebooks.

In [439]:
df_turns_no_outlier.to_pickle('Cleaned_Working_Dataframe.pkl')